# Crewai + Logistics Optimization

#### Refered documentation: https://docs.crewai.com/en/guides/crews/first-crew

In [7]:
from crewai import Agent, Task, Crew, Process
from pydantic import BaseModel
from typing import List, Dict
import os


In [9]:
openai_api_key = "<API KEY>"

os.environ["OPENAI_API_KEY"] = openai_api_key

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [36]:
class AnalysisOutput(BaseModel):
    summary: str
    key_metrics: Dict[str, float]

class StrategyOutput(BaseModel):
    recommendations: List[str]
    expected_benefit: str

In [48]:
logistics_analyst = Agent(
    role="Logistics Analyst",
    goal="Analyze logistics operations (routes, inventory, delays) for given products",
    backstory="An operations researcher who has worked in supply chain optimization.",
    llm="gpt-4o-mini",            
    max_iter=3,            
    max_execution_time=60,  
    verbose=True
)

optimization_strategist = Agent(
    role="Optimization Strategist",
    goal="Propose actionable optimizations based on operations analysis",
    backstory="A strategist with deep experience in logistics cost-reduction and efficiency.",
    llm="gpt-4o-mini",
    max_iter=2,
    max_execution_time=60,
    verbose=True
)


In [50]:
analysis_task = Task(
    description=(
        "Analyze the provided logistics data for the products: {products}. "
        "Use ONLY the supplied data: routes={routes}, inventory={inventory}, "
        "forecast={demand_forecast}, delivery_costs={delivery_costs}. "
        "If any field is missing, say 'Data missing: <field>'. "
        "Compute actual metrics such as route inefficiency, stockout %, "
        "delivery delays, and inventory turnover strictly from the input numbers."
    ),
    agent=logistics_analyst,
    expected_output="A JSON-like dictionary with keys: 'summary', 'key_metrics'."
)

strategy_task = Task(
    description=(
        "Using ONLY the metrics from the analysis report {analysis_task.output}, "
        "develop an optimization strategy. Do NOT invent numbers. "
        "Base recommendations strictly on the provided data. "
    ),
    agent=optimization_strategist,
    context=[analysis_task],
    expected_output="A list of recommendations and a paragraph explaining expected benefits."
)

In [52]:

crew = Crew(
    agents=[logistics_analyst, optimization_strategist],
    tasks=[analysis_task, strategy_task],
    process=Process.sequential,
    verbose=True
)

inputs = {
    "products": ["Laptops", "Mobile Chargers", "Printers"],

    "routes": [
        {"route_id": 1, "planned_km": 120, "actual_km": 150, "delay_hours": 2},
        {"route_id": 2, "planned_km": 90,  "actual_km": 95,  "delay_hours": 0.5},
        {"route_id": 3, "planned_km": 70,  "actual_km": 80,  "delay_hours": 1},
    ],

    "inventory": [
        {"product": "Laptops", "opening": 400, "closing": 300, "sales": 150},
        {"product": "Mobile Chargers", "opening": 900, "closing": 700, "sales": 250},
        {"product": "Printers", "opening": 200, "closing": 180, "sales": 50}
    ],

    "demand_forecast": {
        "Laptops": 160,
        "Mobile Chargers": 300,
        "Printers": 60
    },

    "delivery_costs": {
        "fuel_cost_per_km": 7.5,
        "avg_labor_cost_per_hour": 320
    }
}



In [54]:
result = crew.kickoff(inputs=inputs)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b5ef9548-f6fd-4e0e-b6cc-76f952c1d41b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Logistics Analyst                                                                                       │
│                                                                                                                 │
│  Task: Analyze the provided logistics data for the products: ['Laptops', 'Mobile Chargers', 'Printers']. Use    │
│  ONLY the supplied data: routes=[{'route_id': 1, 'planned_km': 120, 'actual_km': 150, 'delay_hours': 2},        │
│  {'route_id': 2, 'planned_km': 90, 'actual_km': 95, 'delay_hours': 0.5}, {'route_id': 3, 'planned_km': 70,      │
│  'actual_km': 80, 'delay_hours': 1}], inventory=[{'product': 'Laptops', 'opening': 400, 'closing': 300,         │
│  'sales': 150}, {'product': 'Mobile Chargers', 'opening': 900, 'closing': 700, 'sales': 250}, {'product':       │
│  'Printers', 'opening': 200, 'closing': 180, 'sales': 50}], forecast={'Laptops': 160, 'Mobile Chargers': 300,   │
│  'Printers': 60}, delivery_costs={'fuel_cost_per_km': 7.5, 'avg_labor_cost_per_hour': 320}. If any field is     │
│  missing, say 'Data missing: <field>'. Compute actual metrics such as route inefficiency, stockout %, delivery  │
│  delays, and inventory turnover strictly from the input numbers.                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Logistics Analyst                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "summary": {                                                                                                 │
│      "routes": [                                                                                                │
│        {                                                                                                        │
│          "route_id": 1,                                                                                         │
│          "planned_distance_km": 120,                                                                            │
│          "actual_distance_km": 150,                                                                             │
│          "delay_hours": 2,                                                                                      │
│          "inefficiency": 25.0                                                                                   │
│        },                                                                                                       │
│        {                                                                                                        │
│          "route_id": 2,                                                                                         │
│          "planned_distance_km": 90,                                                                             │
│          "actual_distance_km": 95,                                                                              │
│          "delay_hours": 0.5,                                                                                    │
│          "inefficiency": 5.56                                                                                   │
│        },                                                                                                       │
│        {                                                                                                        │
│          "route_id": 3,                                                                                         │
│          "planned_distance_km": 70,                                                                             │
│          "actual_distance_km": 80,                                                                              │
│          "delay_hours": 1,                                                                                      │
│          "inefficiency": 14.29                                                                                  │
│        }                                                                                                        │
│      ],                                                                                                         │
│      "inventory": [                                                                                             │
│        {                                                                                                        │
│          "product": "Laptops",                                                                                  │
│          "opening_stock": 400,                                                                                  │
│          "closing_stock": 300,                         

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Optimization Strategist                                                                                 │
│                                                                                                                 │
│  Task: Using ONLY the metrics from the analysis report {analysis_task.output}, develop an optimization          │
│  strategy. Do NOT invent numbers. Base recommendations strictly on the provided data.                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Optimization Strategist                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Recommendations:**                                                                                           │
│                                                                                                                 │
│  1. **Route Optimization for Route 1**: Analyze the cause of the overage in actual distance (150 km vs.         │
│  planned 120 km) leading to a delay of 2 hours. Implement GPS tracking technology to enforce route adherence    │
│  and identify bottlenecks in traffic conditions.                                                                │
│                                                                                                                 │
│  2. **Reduced Use of Inefficient Routes**: Reformulate routes based on the inefficiency scores, particularly    │
│  focusing on Route 1 (25% inefficiency) and Route 3 (14.29% inefficiency). Consider rerouting or utilizing      │
│  alternative paths for efficiency improvements.                                                                 │
│                                                                                                                 │
│  3. **Enhanced Inventory Management for Laptops**: Given the high stockout percentage of 37.5% for laptops,     │
│  elevate safety stock levels or adjust reorder points to ensure adequate availability. Evaluate real-time       │
│  inventory tracking to optimize order timing.                                                                   │
│                                                                                                                 │
│  4. **Adjustment of Inventory Levels for Mobile Chargers**: With a stockout percentage of 25%, revise           │
│  restocking practices and ensure timely replenishment. Increase engagement with suppliers for better lead       │
│  times.                                                                                                         │
│                                                                                                                 │
│  5. **Streamlined Delivery Schedule**: With a total delivery delay of 3.5 hours, evaluate the scheduling of     │
│  deliveries and incorporate buffer times more effectively to enhance customer satisfaction while maintaining    │
│  efficiency.                                                                                                    │
│                                                                                                                 │
│  6. **Active Monitoring of Inventory Turnover**: The turnover rates, particularly 0.375 for laptops, 0.278 for  │
│  mobile chargers, and 0.25 for printers, suggest a need for regular review of sales forecasts and stock         │
│  replenishment strategies. Utilize sales data to adjust purchasing decisions dynamically.                       │
│                                                                                                                 │
│  **Expected Benefits:**                                                                                         │
│  Implementing these optimization strategies is anticipated to significantly reduce logistical costs and         │
│  improve efficiency throughout operations. By focusing on route optimization and correcting inefficiencies, we  │
│  can expect a lowered overall route inefficiency score,

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ff43c56f-1c7d-4584-b35a-04839f847e5f                                                                     │
│  Agent: Optimization Strategist                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



╭────────────────────────────────────────────────── Execution Traces ──────────────────────────────────────────────────╮
│                                                                                                                      │
│  🔍 Detailed execution traces are available!                                                                         │
│                                                                                                                      │
│  View insights including:                                                                                            │
│    • Agent decision-making process                                                                                   │
│    • Task execution flow and timing                                                                                  │
│    • Tool usage details                                                                                              │
│                              

In [63]:
print("\n===== 📌 RAW FINAL OUTPUT =====\n")
print(result.raw)

print("\n===== 📌 TASKS SUMMARY =====\n")

for task_output in result.tasks_output:
    print(f"--- Task: {task_output.name} ---")
    print(f"Agent: {task_output.agent}")
    print(f"Description: {task_output.description}")
    print("\n[ Raw Output ]\n")
    print(task_output.raw)
    print("\n------------------------------\n")

print("\n===== 📌 INDIVIDUAL SECTIONS =====\n")

analysis = result.tasks_output[0]
strategy = result.tasks_output[1]

print("=== 🔍 ANALYSIS SUMMARY ===")
print(analysis.raw)

print("\n=== 🎯 OPTIMIZATION STRATEGY ===")
print(strategy.raw)

print("\n===== 📊 Token Usage =====\n")
print(result.token_usage)



===== 📌 RAW FINAL OUTPUT =====

**Recommendations:**

1. **Route Optimization for Route 1**: Analyze the cause of the overage in actual distance (150 km vs. planned 120 km) leading to a delay of 2 hours. Implement GPS tracking technology to enforce route adherence and identify bottlenecks in traffic conditions.

2. **Reduced Use of Inefficient Routes**: Reformulate routes based on the inefficiency scores, particularly focusing on Route 1 (25% inefficiency) and Route 3 (14.29% inefficiency). Consider rerouting or utilizing alternative paths for efficiency improvements.

3. **Enhanced Inventory Management for Laptops**: Given the high stockout percentage of 37.5% for laptops, elevate safety stock levels or adjust reorder points to ensure adequate availability. Evaluate real-time inventory tracking to optimize order timing.

4. **Adjustment of Inventory Levels for Mobile Chargers**: With a stockout percentage of 25%, revise restocking practices and ensure timely replenishment. Increase e